# 1.1 Imports

In [55]:
# import libraries
import numpy as np
import pandas as pd

import urllib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from PIL import Image
import re

In [56]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [57]:
# open web browser
driver = webdriver.Chrome('C:/Users/Leung/Desktop/chromedriver.exe')

In [58]:
# navigate webpage
driver.get('https://www.calvinklein.com/hk/en/women-apparel-activewear/')

# 1.2 Extracting Links

In [59]:
# create text file
f = open("activewear_links.txt", "x")

In [60]:
# get all product elements

product_elements = driver.find_elements_by_xpath('//a[@class="name-link"]')

In [62]:
# get all product links

f = open("activewear_links.txt", "a")

for product_element in product_elements[:10]:

    print(product_element.get_attribute('href'))

    f.write(product_element.get_attribute('href'))
    f.write('\n')

f.close()

https://www.calvinklein.com/hk/en/lace-lightwear-jacket-4WT1O553.html?dwvar_4WT1O553_color=BRIGHT%20WHITE
https://www.calvinklein.com/hk/en/back-v-lace-short-sleeve-dress-4WS1D981.html?dwvar_4WS1D981_color=CK%20BLACK
https://www.calvinklein.com/hk/en/lace-fitsense-mesh-hem-leggings-4WT1L784.html?dwvar_4WT1L784_color=CK%20BLACK
https://www.calvinklein.com/hk/en/logo-short-sleeve-tee-4WT1K131.html?dwvar_4WT1K131_color=BRIGHT%20WHITE
https://www.calvinklein.com/hk/en/active-icon-front-zip-sports-bra-4WS1K131.html?dwvar_4WS1K131_color=CK%20BLACK
https://www.calvinklein.com/hk/en/active-icon-7%2F8-fit-sense-leggings-4WS1L649.html?dwvar_4WS1L649_color=CK%20BLACK
https://www.calvinklein.com/hk/en/active-icon-hooded-sweat-jacket-4WS1J474.html?dwvar_4WS1J474_color=CK%20BLACK
https://www.calvinklein.com/hk/en/ck-one-tanga-bikini-KW01435.html?dwvar_KW01435_color=BLACK%20CUT%20OUT%20PRINT
https://www.calvinklein.com/hk/en/active-icon-full-length-all-over-print-leggings-4WS1L650.html?dwvar_4WS1L650

In [63]:
# load all links

links = []
f = open('activewear_links.txt','r')
for link in f.read().split():
    links.append(link)
links

['https://www.calvinklein.com/hk/en/lace-lightwear-jacket-4WT1O553.html?dwvar_4WT1O553_color=BRIGHT%20WHITE',
 'https://www.calvinklein.com/hk/en/back-v-lace-short-sleeve-dress-4WS1D981.html?dwvar_4WS1D981_color=CK%20BLACK',
 'https://www.calvinklein.com/hk/en/lace-fitsense-mesh-hem-leggings-4WT1L784.html?dwvar_4WT1L784_color=CK%20BLACK',
 'https://www.calvinklein.com/hk/en/logo-short-sleeve-tee-4WT1K131.html?dwvar_4WT1K131_color=BRIGHT%20WHITE',
 'https://www.calvinklein.com/hk/en/active-icon-front-zip-sports-bra-4WS1K131.html?dwvar_4WS1K131_color=CK%20BLACK',
 'https://www.calvinklein.com/hk/en/active-icon-7%2F8-fit-sense-leggings-4WS1L649.html?dwvar_4WS1L649_color=CK%20BLACK',
 'https://www.calvinklein.com/hk/en/active-icon-hooded-sweat-jacket-4WS1J474.html?dwvar_4WS1J474_color=CK%20BLACK',
 'https://www.calvinklein.com/hk/en/ck-one-tanga-bikini-KW01435.html?dwvar_KW01435_color=BLACK%20CUT%20OUT%20PRINT',
 'https://www.calvinklein.com/hk/en/active-icon-full-length-all-over-print-leg

# 1.3 Extracting Product Details

In [64]:
# testing
testing_link = 'https://www.calvinklein.com/hk/en/lace-lightwear-jacket-4WT1O553.html?dwvar_4WT1O553_color=BRIGHT%20WHITE'

In [65]:
# navigate webpage
driver.get(testing_link)

In [66]:
# get product name

name = driver.find_element_by_tag_name('h1')
name = name.text

In [67]:
name

'LACE LIGHTWEAR JACKET'

In [71]:
# get product price

price = driver.find_element_by_xpath('//span[@class="price-standard"]')
price = price.text

In [72]:
price

'HKD 1,290.00'

In [74]:
# size
img_width,img_height = 300,300

# get all images
images = driver.find_elements_by_xpath('//img[@class="primary-image cloudzoom"]')

# product image at index 0
img = images[0]

# 'src' = get image source
src = img.get_attribute('src')

# download image
urllib.request.urlretrieve(src, f'image/{name}.png')

# resize image (smaller size)
ori_img = Image.open(f'image/{name}.png')
resize_img = ori_img.resize((img_width,img_height))
resize_img.save(f'image/{name}.png')

img_file = f'image/{name}.png'

In [ ]:
# END